In [1]:
from pathlib import Path
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import pandas as pd

In [2]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from torch import optim
import time
import math

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device

device(type='cuda', index=0)

In [4]:
class SkeletonData:
    def __init__(self, dpath):
        self.data_path = dpath + '/nturgb+d_skeletons/'
        self.broken_files_path = dpath + '/NTU_RGBD_samples_with_missing_skeletons.txt'
        self.training_subjects = list(range(0, 28))
        self.training_cameras = [1, 2, 3]
        self.training_classes = [8, 10, 21, 22, 23, 25, 26, 27, 43, 52]
        self.working_files_with_labels, self.action_classes = self.read_data(self.data_path, self.broken_files_path)
    

    def read_data(self, data_path, broken_files_path):
        labels = []
        files = []
        action_classes = {}
        counter = 0
        files_counter = {}

        with open(broken_files_path, 'r') as f:
            broken_files = f.read().split("\n")

        raw_files = os.listdir(data_path)
        num_frames = 0

        for filename in raw_files:
            if filename not in broken_files:
                action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                if action_class in self.training_classes and camera_id in self.training_cameras:  #and subject_id in training_subjects:
                    if action_class in action_classes:
                        if files_counter[action_class] < 120:
                            files.append([filename,action_classes[action_class]])
                            files_counter[action_class] = files_counter[action_class] + 1
                    else:
                        action_classes.update({action_class : counter})
                        files_counter.update({action_class : 1})
                        counter+=1
                        files.append([filename,action_classes[action_class]])
    #                     labels.append([action_class])
#         print("action classes: ", action_classes)
#         print("action files: ", files_counter)

        return files, action_classes
    
    
    @staticmethod
    def read_skeleton_filter(file):
        with open(file, 'r') as f:
            skeleton_sequence = {}
            skeleton_sequence['numFrame'] = int(f.readline())
            skeleton_sequence['frameInfo'] = []
            for t in range(skeleton_sequence['numFrame']):
                frame_info = {}
                frame_info['numBody'] = int(f.readline())
                frame_info['bodyInfo'] = []

                for m in range(frame_info['numBody']):
                    body_info = {}
                    body_info_key = [
                        'bodyID', 'clipedEdges', 'handLeftConfidence',
                        'handLeftState', 'handRightConfidence', 'handRightState',
                        'isResticted', 'leanX', 'leanY', 'trackingState'
                    ]
                    body_info = {
                        k: float(v)
                        for k, v in zip(body_info_key, f.readline().split())
                    }
                    body_info['numJoint'] = int(f.readline())
                    body_info['jointInfo'] = []
                    for v in range(body_info['numJoint']):
                        joint_info_key = [
                            'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                            'orientationW', 'orientationX', 'orientationY',
                            'orientationZ', 'trackingState'
                        ]
                        joint_info = {
                            k: float(v)
                            for k, v in zip(joint_info_key, f.readline().split())
                        }
                        body_info['jointInfo'].append(joint_info)
                    frame_info['bodyInfo'].append(body_info)
                skeleton_sequence['frameInfo'].append(frame_info)

        return skeleton_sequence
    
    

    def read_xyz(self, file, max_body=1, num_joint=25):
        seq_info = self.read_skeleton_filter(file)
        data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
        for n, f in enumerate(seq_info['frameInfo']):
            for m, b in enumerate(f['bodyInfo']):
                for j, v in enumerate(b['jointInfo']):
                    if m < max_body and j < num_joint:
                        data[m, n, j, :] = [v['x'], v['y'], v['z']]

                    else:
                        pass

        return data
    
    

    def create_coords_blocks(self, test_file, chonk_len):   
        frame_counter = 0
        new_labels = []
        new_frames = []
        blocks = []

        test_frames = self.read_xyz(str(self.data_path) + test_file[0])[0]
        label = test_file[1]
        slice_len = chonk_len * int(len(test_frames)/chonk_len)


        for index in range(len(test_frames[:slice_len])):
            frame_counter += 1
            new_frames.append(test_frames[index].flatten())
            if frame_counter == chonk_len:
                frame_counter = 0
                blocks.append(np.array(new_frames))
                new_labels = new_labels + [label]
                new_frames = []


        return blocks, new_labels
    

    def create_dataset(self, chonk_len):
        data = []
        labels = []
        ##########################################################################
        numbers = {x: 0 for x in range(len(self.action_classes))} #####
        ##################################################################
        for file in self.working_files_with_labels:
            frames_blocks, label = self.create_coords_blocks(file, chonk_len)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] = numbers[label[0]] + len(label)
                data = data + frames_blocks
                labels = labels + label
        data_np = np.asarray(data)
        labels_np = np.asarray(labels)

        data_sq = data_np.reshape(len(data_np), -1)
        test_data = pd.DataFrame(data_sq)
        test_labels = pd.DataFrame(labels_np)
        test_data['labels'] = test_labels
        
        return test_data

In [5]:
skeletons45 = SkeletonData('D:/GeekBrains/pytorch/les05/data').create_dataset(45)
skeletons45.head()

,0,1,2,3,4,5,6,7,8,9,...,3366,3367,3368,3369,3370,3371,3372,3373,3374,labels
0,-0.080141,0.135073,3.691227,-0.058513,0.396866,3.606265,-0.037811,0.652906,3.509790,-0.066903,...,-0.066586,-0.194186,3.880914,0.103226,-0.183070,3.717531,0.180819,-0.130425,3.724300,0
1,-0.035358,0.192430,3.681518,-0.021036,0.422171,3.606487,-0.007186,0.646429,3.520689,-0.042399,...,-0.247820,0.496547,3.406900,-0.056666,0.495373,3.269386,-0.050549,0.490788,3.258667,1
2,0.430001,0.024285,4.098490,0.424111,0.227365,4.005508,0.417157,0.427353,3.902021,0.395301,...,0.098534,0.515498,3.738000,0.243639,0.441711,3.635301,0.292146,0.468907,3.595111,2
3,0.399282,-0.026338,4.099298,0.382165,0.177244,4.002089,0.359935,0.376536,3.893836,0.338453,...,0.300487,0.700893,3.694455,0.535627,0.814146,3.868758,0.530421,0.797046,3.864750,3
4,0.449151,0.001842,4.170505,0.412885,0.192257,4.103466,0.375976,0.379683,4.026225,0.401664,...,0.723439,0.436599,4.077281,0.535588,0.585105,4.202623,0.536817,0.559229,4.186821,3


In [6]:
skeletons45.shape

(1351, 3376)

In [7]:
skeletons45.to_csv("skels45.csv", index = False)

### dataset

In [8]:
LABELS = {0: "sitting down", 1: "clapping", 2:  "take off a hat/cap", 3: "cheer up", 4: "hand waving", 5: "reach into pocket", 6: "hopping", 7:  "jump up", 8: "falling", 9: "pushing other person"}

In [9]:
class Skeleton_Dataset(Dataset):
    def __init__(self, file_path, frames, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        self.labels = self.data.iloc[:,-1]
        self.frames = frames



    def __len__(self):
        return len(self.data) 
    
    
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(self.frames,25*3)
        label = self.labels[idx]
        if self.transform != None:
            item = transform(item)
            
        

        return (item, label)

In [10]:
dataset = Skeleton_Dataset(file_path = "skels45.csv", frames = 45, transform=None)

In [11]:
skel, lab = dataset.__getitem__(41)
skel, lab = dataset[41]

In [12]:
LABELS[lab]

'hand waving'

In [13]:
skel.shape

(45, 75)

In [14]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.75*len(dataset)), int(0.25*len(dataset)+1)])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

In [15]:
int(0.75*len(dataset)), int(0.25*len(dataset))

(1013, 337)

In [16]:
sample = iter(train_loader).next()

In [17]:
sample[0].size(), sample[1].size()

(torch.Size([16, 45, 75]), torch.Size([16]))

### model

In [18]:
class LSTM_net(nn.Module):
    # Для того, что бы инициализировать LSTM нам нужно указать:
    # input_dim - размерность входного тензора. тензор входит в формате (seq_len, batch, input_dim)
    # (batch_size, seq, inp_dim) - if batch_first=True
    # hidden_dim - размерность вектора состояния h
    # output_dim - размерность выхода
    # layer_num - количество скрытых слоев в сети
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [19]:
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [20]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
#     print(output.topk(5))
    return LABELS[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [21]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(600):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 2.2961  / falling ✗ (hopping)
epoch : 7 iter : 52 (0m 16s) 2.2837  / reach into pocket ✗ (sitting down)
epoch : 15 iter : 40 (0m 32s) 2.1810  / reach into pocket ✗ (hopping)
epoch : 23 iter : 28 (0m 46s) 2.3644  / falling ✓
epoch : 31 iter : 16 (1m 1s) 1.6294  / take off a hat/cap ✗ (hand waving)
epoch : 39 iter : 4 (1m 17s) 1.6062  / hopping ✓
epoch : 46 iter : 56 (1m 32s) 1.6428  / take off a hat/cap ✗ (jump up)
epoch : 54 iter : 44 (1m 47s) 1.5210  / cheer up ✗ (hand waving)
epoch : 62 iter : 32 (2m 3s) 1.0790  / take off a hat/cap ✗ (cheer up)
epoch : 70 iter : 20 (2m 18s) 1.4915  / hand waving ✓
epoch : 78 iter : 8 (2m 34s) 1.1317  / hopping ✓
epoch : 85 iter : 60 (2m 49s) 1.1864  / cheer up ✓
epoch : 93 iter : 48 (3m 5s) 0.7860  / pushing other person ✓
epoch : 101 iter : 36 (3m 20s) 0.7731  / clapping ✗ (take off a hat/cap)
epoch : 109 iter : 24 (3m 36s) 0.8957  / hopping ✓
epoch : 117 iter : 12 (3m 52s) 0.6957  / sitting down ✓
epoch : 125 iter : 0 (4

In [22]:
def evaluate_result(model, test_loader):
    total = 0
    right = 0
    counter = 0

    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            counter = counter + 1
            inputs, labels = data[0].to(device), data[1].to(device)  
            output = model(inputs.float())
            guess, guess_i = categoryFromOutput(output)
            category = LABELS[int(labels[0])]

            if guess == category:
                right = right + 1


    return 100 * right / counter

In [23]:
evaluate_result(rnn, test_loader)

76.33136094674556

In [24]:
res = {}
res['l2_f45'] = evaluate_result(rnn, test_loader)
pd.DataFrame.from_dict(res, orient='index', columns = ['acc_test'])

,acc_test
l2_f45,76.331361


#### layers

In [25]:
for l in [4, 6, 8]:
    model = LSTM_net(n_joints,n_hidden,n_categories,l)
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    learning_rate = 0.0007
    optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9)

    all_losses = []
    start = time.time()
    counter = 0
    for epoch in range(600):  
        current_loss = 0
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):

            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()

            output = model(inputs.float())
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step() 


            current_loss += loss.item()
            category = LABELS[int(labels[0])]

            if counter % 500 == 0:
                guess, guess_i = categoryFromOutput(output)
                correct = '✓' if guess == category else '✗ (%s)' % category
                print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))


            counter = counter + 1
        if counter % 100 == 0:
            all_losses.append(current_loss / 25)
            current_loss = 0
            
    
    res['l'+ str(l) + '_f45'] = evaluate_result(model, test_loader)

epoch : 0 iter : 0 (0m 0s) 2.2986  / hopping ✓
epoch : 7 iter : 52 (0m 16s) 2.2791  / hopping ✗ (pushing other person)
epoch : 15 iter : 40 (0m 33s) 2.2943  / hopping ✗ (cheer up)
epoch : 23 iter : 28 (0m 50s) 2.2862  / reach into pocket ✗ (take off a hat/cap)
epoch : 31 iter : 16 (1m 7s) 2.3032  / reach into pocket ✗ (hand waving)
epoch : 39 iter : 4 (1m 24s) 2.3242  / reach into pocket ✓
epoch : 46 iter : 56 (1m 41s) 2.2585  / reach into pocket ✗ (sitting down)
epoch : 54 iter : 44 (1m 58s) 2.3193  / reach into pocket ✗ (cheer up)
epoch : 62 iter : 32 (2m 15s) 2.2987  / reach into pocket ✗ (falling)
epoch : 70 iter : 20 (2m 32s) 2.2798  / reach into pocket ✗ (hand waving)
epoch : 78 iter : 8 (2m 50s) 2.2795  / reach into pocket ✗ (cheer up)
epoch : 85 iter : 60 (3m 9s) 2.2805  / reach into pocket ✗ (falling)
epoch : 93 iter : 48 (3m 26s) 2.2977  / reach into pocket ✗ (cheer up)
epoch : 101 iter : 36 (3m 43s) 2.2490  / reach into pocket ✗ (falling)
epoch : 109 iter : 24 (4m 0s) 2.3638

epoch : 375 iter : 0 (14m 56s) 2.3395  / reach into pocket ✓
epoch : 382 iter : 52 (15m 15s) 2.3138  / reach into pocket ✓
epoch : 390 iter : 40 (15m 33s) 2.2417  / reach into pocket ✓
epoch : 398 iter : 28 (15m 52s) 2.2933  / reach into pocket ✗ (cheer up)
epoch : 406 iter : 16 (16m 11s) 2.3233  / reach into pocket ✗ (pushing other person)
epoch : 414 iter : 4 (16m 29s) 2.2652  / reach into pocket ✓
epoch : 421 iter : 56 (16m 50s) 2.2812  / reach into pocket ✗ (take off a hat/cap)
epoch : 429 iter : 44 (17m 9s) 2.3281  / reach into pocket ✗ (sitting down)
epoch : 437 iter : 32 (17m 27s) 2.2906  / reach into pocket ✗ (cheer up)
epoch : 445 iter : 20 (17m 45s) 2.2815  / reach into pocket ✗ (sitting down)
epoch : 453 iter : 8 (18m 4s) 2.2926  / reach into pocket ✗ (hopping)
epoch : 460 iter : 60 (18m 22s) 2.2956  / reach into pocket ✓
epoch : 468 iter : 48 (18m 41s) 2.2479  / reach into pocket ✗ (take off a hat/cap)
epoch : 476 iter : 36 (18m 59s) 2.2563  / reach into pocket ✗ (cheer up)

In [26]:
pd.DataFrame.from_dict(res, orient='index', columns = ['acc_test'])

,acc_test
l2_f45,76.331361
l4_f45,74.556213
l6_f45,10.355030
l8_f45,10.355030


С увеличением количества модулей время на обработку растет, а модель перестает обучаться и предсказывает один и тот же класс

#### frames

In [27]:
skeletons25 = SkeletonData('D:/GeekBrains/pytorch/les05/data').create_dataset(25)
skeletons65 = SkeletonData('D:/GeekBrains/pytorch/les05/data').create_dataset(65)

In [28]:
skeletons25.to_csv("skels25.csv", index = False)
skeletons65.to_csv("skels65.csv", index = False)

In [29]:
dataset25 = Skeleton_Dataset(file_path = "skels25.csv", frames = 25, transform=None)
dataset65 = Skeleton_Dataset(file_path = "skels65.csv", frames = 65, transform=None)

In [30]:
skeletons25.shape, skeletons65.shape

((1517, 1876), (912, 4876))

In [31]:
skel, lab = dataset25.__getitem__(41)
skel, lab = dataset25[41]
skel.shape

(25, 75)

In [32]:
skel, lab = dataset65.__getitem__(41)
skel, lab = dataset65[41]
skel.shape

(65, 75)

In [33]:
train_dataset25, test_dataset25 = torch.utils.data.random_split(dataset25, [int(0.75*len(dataset25)), int(0.25*len(dataset25)+1)])
train_loader25 = DataLoader(train_dataset25, batch_size = 16, shuffle=True)
test_loader25 = DataLoader(test_dataset25, batch_size = 1, shuffle=True)

train_dataset65, test_dataset65 = torch.utils.data.random_split(dataset65, [int(0.75*len(dataset65)), int(0.25*len(dataset65))])
train_loader65 = DataLoader(train_dataset25, batch_size = 16, shuffle=True)
test_loader65 = DataLoader(test_dataset25, batch_size = 1, shuffle=True)

In [34]:
sample = iter(train_loader25).next()
sample[0].size(), sample[1].size()

(torch.Size([16, 25, 75]), torch.Size([16]))

In [35]:
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn2 = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn2.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [36]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn2.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(600):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader25, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn2(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 2.3050  / jump up ✓
epoch : 6 iter : 68 (0m 10s) 2.2917  / falling ✓
epoch : 13 iter : 64 (0m 22s) 2.2820  / falling ✗ (pushing other person)
epoch : 20 iter : 60 (0m 33s) 2.2654  / cheer up ✓
epoch : 27 iter : 56 (0m 43s) 2.1686  / falling ✗ (sitting down)
epoch : 34 iter : 52 (0m 52s) 2.1436  / hopping ✗ (clapping)
epoch : 41 iter : 48 (1m 2s) 1.8029  / hopping ✓
epoch : 48 iter : 44 (1m 11s) 2.0811  / falling ✓
epoch : 55 iter : 40 (1m 21s) 1.6962  / hopping ✗ (pushing other person)
epoch : 62 iter : 36 (1m 30s) 1.8665  / falling ✓
epoch : 69 iter : 32 (1m 40s) 1.6647  / take off a hat/cap ✗ (hand waving)
epoch : 76 iter : 28 (1m 49s) 1.6218  / hand waving ✗ (jump up)
epoch : 83 iter : 24 (1m 59s) 2.1850  / reach into pocket ✗ (sitting down)
epoch : 90 iter : 20 (2m 8s) 1.4607  / reach into pocket ✗ (jump up)
epoch : 97 iter : 16 (2m 17s) 1.7233  / reach into pocket ✗ (pushing other person)
epoch : 104 iter : 12 (2m 27s) 2.2826  / hand waving ✗ (falling)
e

In [37]:
res['l2_f25'] = evaluate_result(rnn2, test_loader25)
pd.DataFrame.from_dict(res, orient='index', columns = ['acc_test'])

,acc_test
l2_f45,76.331361
l4_f45,74.556213
l6_f45,10.355030
l8_f45,10.355030
l2_f25,66.842105


In [38]:
rnn3 = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn3.to(device)

criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn3.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(600):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader65, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn3(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 2.3102  / take off a hat/cap ✗ (cheer up)
epoch : 6 iter : 68 (0m 11s) 2.2906  / falling ✗ (take off a hat/cap)
epoch : 13 iter : 64 (0m 21s) 2.2903  / falling ✗ (jump up)
epoch : 20 iter : 60 (0m 31s) 2.2267  / falling ✗ (sitting down)
epoch : 27 iter : 56 (0m 41s) 2.1654  / falling ✗ (jump up)
epoch : 34 iter : 52 (0m 50s) 2.0235  / cheer up ✗ (pushing other person)
epoch : 41 iter : 48 (1m 0s) 2.1146  / cheer up ✓
epoch : 48 iter : 44 (1m 10s) 2.0012  / pushing other person ✗ (clapping)
epoch : 55 iter : 40 (1m 20s) 1.7791  / take off a hat/cap ✗ (hand waving)
epoch : 62 iter : 36 (1m 29s) 1.5001  / cheer up ✓
epoch : 69 iter : 32 (1m 39s) 1.4549  / hopping ✓
epoch : 76 iter : 28 (1m 49s) 1.5049  / pushing other person ✗ (reach into pocket)
epoch : 83 iter : 24 (1m 58s) 1.3696  / pushing other person ✓
epoch : 90 iter : 20 (2m 8s) 1.3566  / hopping ✗ (reach into pocket)
epoch : 97 iter : 16 (2m 18s) 1.2453  / cheer up ✓
epoch : 104 iter : 12 (2m 27s) 1.391

In [39]:
res['l2_f65'] = evaluate_result(rnn3, test_loader65)
pd.DataFrame.from_dict(res, orient='index', columns = ['acc_test'])

,acc_test
l2_f45,76.331361
l4_f45,74.556213
l6_f45,10.355030
l8_f45,10.355030
l2_f25,66.842105
l2_f65,67.105263


Первоначальные настройки дали самый лучший результат по точности